In [1]:
import numpy as np
import scipy.optimize
from sklearn.linear_model import LinearRegression

In [2]:
class Dynamics:
    pass

In [3]:
hw7dynamics=Dynamics()
hw7dynamics.S0 = 1
hw7dynamics.r = 0.03
hw7dynamics.sigma = 0.20

In [4]:
class Contract:
    pass

In [5]:
hw7contract=Contract() 
hw7contract.K = 1.1 
hw7contract.T = 4 

In [6]:
class MC:
    pass

In [7]:
hw7MC=MC()
hw7MC.M = 100000 # Number of paths  
hw7MC.N = 4     # Number of time periods  
hw7MC.seed = 0  # Seeding the random number generator with a specified number helps make the calculations reproducible

hw7MC.algorithm = 'value'   
#'value' for Value-based approach (Longstaff-Schwartz) -- problem 1
#'policy' for Policy optimization -- problem 2

In [8]:
# for Policy optimization approach, problem 2
#
# If b<<0 then this function essentially returns nearly 1 if X<a, or nearly 0 if X>a
# but with some smoothing of the discontinuity, using a sigmoid function, to help the optimizer

def softExercise(X,a,b):
    return 1/(1+np.exp(-b*(X-a)))

In [9]:
# for Policy optimization approach, problem 2

def negofMCaverageOfExpectedPayouts(coefficients, x, exercisePayoff, continuationPayoff):

    p = softExercise(x,*coefficients)    

    # p and exercisePayoff and continuationPayoff are all length-M arrays

    return -(p*exercisePayoff + (1-p)*continuationPayoff).mean()

## You fill in, what to return.  It should be the negative of the expression inside the max() on the homework sheet.
## Need to take the negative because we are calling "minimize" but we want to do _maximization_

In [10]:
def pricer_americanPut_GBM_LSM(contract,dynamics,MC):

    np.random.seed(MC.seed)  #seed the random number generator
    
    r=dynamics.r
    sigma=dynamics.sigma
    S0=dynamics.S0

    K=contract.K
    T=contract.T

    N=MC.N
    M=MC.M
    dt=T/N

    Z = np.random.randn(M, N)
    
    paths = S0*np.exp((r-sigma**2/2)*dt*np.tile(np.arange(1,N+1),(M,1))+sigma*np.sqrt(dt)*np.cumsum(Z,axis=1))
    
    payoffDiscounted = np.maximum(0,K-paths[:,-1])
    #This is the payoff (cashflow) along each path,
    #discounted to time nn (for nn=N,N-1,...)
    #It corresponds to the far right-hand column in each page of the
    #Excel worksheet
    #I'm initializing it for time nn=N.

    #You could make payoffDiscounted
    #to be a matrix because it depends on nn.
    #But I will just reuse a 1-dimensional array,
    #by overwriting the time nn+1 entries at time nn.        

    for nn in np.arange(N-1,0,-1):
        continuationPayoffDiscounted = np.exp(-r*dt)*payoffDiscounted
        # This is the CONTINUATION payoff (cashflow) along each path,
        # discounted to time nn (for nn=N-1,N-2,...)
        # It corresponds to the blue column in each page of the Excel worksheet
        # Note that payoffDiscounted comes from the previous iteration 
        # -- which was at time nn+1.  So now we discount back to time nn.

        X=paths[:,nn-1]               
        exerciseValue = K-X
        
        if MC.algorithm == 'value': 
            # This is the value function (Longstaff-Schwartz) approach.  For problem 1

            basisfunctions = np.stack((np.ones(M), X, X**2),axis = -1)
            # FILL THIS IN.  You may use np.stack
                    # This will be an M-by-3 array containing the basis functions (Same ones as L7.8-7.9, and Excel)
            reg = LinearRegression(fit_intercept = False).fit(basisfunctions, continuationPayoffDiscounted)
            
            coefficients = reg.coef_  
                    # This will be an array of 3 estimated "betas".
            
            estimatedContinuationValue = reg.predict(basisfunctions)# FILL THIS IN with an array of length M. 
                    # This is similar to the Red column in Excel
            
            whichPathsToExercise = (exerciseValue >= np.maximum(estimatedContinuationValue,0))
                    #This is a length-M array of Booleans
        
        elif MC.algorithm == 'policy':
            # This is the policy optimization approach to reinforcement learning.  For problem 2
            
            (a_opt,b_opt) = scipy.optimize.minimize(
                negofMCaverageOfExpectedPayouts,(0,0),args=(X,exerciseValue,continuationPayoffDiscounted),method='Nelder-Mead').x
                #Chose Nelder-Mead optimizer because it is generating reasonable results with minimal coding effort
                #But gradient methods, done properly, usually run faster
    
            whichPathsToExercise = np.where(softExercise(X,a_opt,b_opt)>=0.5,1,0) * np.where(exerciseValue>0,1,0)
                #FILL THIS IN, using the right-hand side of the last equation on the homework sheet 
                #This obtains the hard exercise decision from the optimized soft exercise function
                #It should be a length-M array of Booleans (as it was in the "value" approach.  
                #But here it comes from the softExercise function)

        else:
            raise ValueError('Unknown algorithm type')
        
        
        payoffDiscounted[whichPathsToExercise] = exerciseValue[whichPathsToExercise] # FILL THIS IN -- see the "discounted cashflow along path" column in Excel 
        payoffDiscounted[np.logical_not(whichPathsToExercise)] = continuationPayoffDiscounted[np.logical_not(whichPathsToExercise)]# FILL THIS IN -- see the "discounted cashflow along path" column in Excel

    # The time-0 calculation needs no regression
    continuationPayoffDiscounted = np.exp(-r*dt)*payoffDiscounted;
    estimatedContinuationValue = np.mean(continuationPayoffDiscounted);
    putprice = max(K-S0,estimatedContinuationValue);
        
    return(putprice)
        

## Part 1 results

In [11]:
pricer_americanPut_GBM_LSM(hw7contract,hw7dynamics,hw7MC)

0.16223563140067124

## Part 2 results

In [12]:
hw7MC.algorithm = 'policy'
pricer_americanPut_GBM_LSM(hw7contract,hw7dynamics,hw7MC)

<ipython-input-8-4b7a501e3e82>:7: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-b*(X-a)))


0.15303872363875284

In [4]:
from scipy.stats import norm 
import numpy as np 

def call_price(d):
    return 100*(norm.cdf(d)-norm.cdf(-d))

target = 6.20

low = 0
high = 0.5
mid = (low+high)/2

while abs(call_price(mid)-target) > 1e-10:
    
    if call_price(mid) > target:
        high = mid
        mid = (low + high)/2 
        
    else:
        low = mid
        mid = (low + high)/2
        
print(mid)
print(call_price(mid))

0.07778384164703311
6.200000000009381


In [6]:
s = (mid*2)/np.sqrt(0.5)
s

0.2200059275814308

In [7]:
r = (s**2)/2-0.1*0.25**2-0.15*0.32**2
r

0.0025913040854828855

In [8]:
np.sqrt(4*r)

0.10180970652119346

In [9]:
np.sqrt(2*(0.1*0.25**2+0.15*0.32**2+0.25*0.1018**2))

0.22000368178737373

In [10]:
def d(s,t):
    return 0.5*s*np.sqrt(t)

In [11]:
call_price(d(0.25,0.1))

3.1530945127879573

In [12]:
call_price(d(0.294,0.25))

5.859175592360261

In [13]:
s=0.36
r=(s**2)/2-0.1*0.25**2
r

0.05855

In [16]:
b = np.sqrt(r/0.15)
b

0.6247666230948428

In [17]:
a = 0.25
np.sqrt(2*(0.1*a**2+0.15*b**2))

0.36

In [18]:
np.sqrt(4*(0.1*a**2+0.15*b**2))

0.5091168824543142